In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

In [2]:
# Hücre 2: Veriyi oku ve 80.000 satır örnekle

df = pd.read_csv("crime_data_clean_azra.csv")

print("Orijinal satır sayısı:", df.shape[0])

# Bilgisayarı çok yormamak için 80.000 satır aldım.
df = df.sample(n=80000, random_state=42)

print("Kullanılan satır sayısı:", df.shape[0])
df.head()

Orijinal satır sayısı: 995231
Kullanılan satır sayısı: 80000


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,LOCATION,LAT,LON
75319,201408407,2020-03-17,2020-03-16,700,14,Pacific,1416,1,310,BURGLARY,...,501.0,SINGLE FAMILY DWELLING,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,310.0,3300 MOORE ST,34.0100,-118.4497
655426,230125961,2023-12-09,2023-12-09,1155,1,Central,138,1,330,BURGLARY FROM VEHICLE,...,122.0,"VEHICLE, PASSENGER/TRUCK",-1.0,UNKNOWN,IC,Invest Cont,330.0,600 E 3RD ST,34.0461,-118.2395
652607,230808162,2023-04-11,2023-04-11,2051,8,West LA,897,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",-1.0,UNKNOWN,IC,Invest Cont,745.0,3300 S BEVERLY DR,34.0314,-118.3988
80531,201311531,2020-05-27,2020-05-27,1800,13,Newton,1375,2,624,BATTERY - SIMPLE ASSAULT,...,104.0,DRIVEWAY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,624.0,1400 E 55TH ST,33.9930,-118.2499
858965,230114610,2023-06-21,2023-06-21,800,1,Central,192,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,122.0,"VEHICLE, PASSENGER/TRUCK",-1.0,UNKNOWN,IC,Invest Cont,745.0,200 VENICE BL,34.0347,-118.2646


In [3]:
# Hücre 3: Tarih ve saat kolonlarından ek feature üretimi

# --- Tarih kolonu ---
date_col_candidates = ['DATE OCC', 'Date', 'OCC_DATE']
date_col = None

for c in date_col_candidates:
    if c in df.columns:
        date_col = c
        break

if date_col is not None:
    df['DATE_OCC_DT'] = pd.to_datetime(df[date_col], errors='coerce')
    df['year'] = df['DATE_OCC_DT'].dt.year
    df['month'] = df['DATE_OCC_DT'].dt.month
    df['dayofweek'] = df['DATE_OCC_DT'].dt.dayofweek  # 0 = Pazartesi
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
else:
    print("Uyarı: Tarih kolonu bulunamadı, year/month/dayofweek/is_weekend üretilmedi.")

# --- Saat kolonu ---
time_col_candidates = ['TIME OCC', 'Time', 'OCC_TIME']
time_col = None

for c in time_col_candidates:
    if c in df.columns:
        time_col = c
        break

if time_col is not None:
    time_num = pd.to_numeric(df[time_col], errors='coerce')
    df['hour'] = (time_num // 100).astype('Int64')  # nullable integer
else:
    print("Uyarı: Saat kolonu bulunamadı, hour üretilmedi.")

df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Crm Cd 1,LOCATION,LAT,LON,DATE_OCC_DT,year,month,dayofweek,is_weekend,hour
75319,201408407,2020-03-17,2020-03-16,700,14,Pacific,1416,1,310,BURGLARY,...,310.0,3300 MOORE ST,34.0100,-118.4497,2020-03-16,2020,3,0,0,7
655426,230125961,2023-12-09,2023-12-09,1155,1,Central,138,1,330,BURGLARY FROM VEHICLE,...,330.0,600 E 3RD ST,34.0461,-118.2395,2023-12-09,2023,12,5,1,11
652607,230808162,2023-04-11,2023-04-11,2051,8,West LA,897,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,745.0,3300 S BEVERLY DR,34.0314,-118.3988,2023-04-11,2023,4,1,0,20
80531,201311531,2020-05-27,2020-05-27,1800,13,Newton,1375,2,624,BATTERY - SIMPLE ASSAULT,...,624.0,1400 E 55TH ST,33.9930,-118.2499,2020-05-27,2020,5,2,0,18
858965,230114610,2023-06-21,2023-06-21,800,1,Central,192,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,745.0,200 VENICE BL,34.0347,-118.2646,2023-06-21,2023,6,2,0,8


In [4]:
# Hücre 4: Gereksiz kolonları at

drop_cols = [
    'DR_NO',
    'DATE_OCC_DT',
    'LOCATION',
    'Crm Cd Desc',
    'Weapon Desc',
    'Status Desc',
    'AREA NAME',
    'Date Rptd',
    'Mocodes'
]

for c in drop_cols:
    if c in df.columns:
        df = df.drop(columns=[c])

print("Drop sonrası kolon sayısı:", df.shape[1])
df.head()

Drop sonrası kolon sayısı: 21


,DATE OCC,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Vict Sex,Vict Descent,Premis Cd,...,Weapon Used Cd,Status,Crm Cd 1,LAT,LON,year,month,dayofweek,is_weekend,hour
75319,2020-03-16,700,14,1416,1,310,33.0,M,W,501.0,...,500.0,IC,310.0,34.0100,-118.4497,2020,3,0,0,7
655426,2023-12-09,1155,1,138,1,330,24.0,F,W,122.0,...,-1.0,IC,330.0,34.0461,-118.2395,2023,12,5,1,11
652607,2023-04-11,2051,8,897,2,745,68.0,F,B,502.0,...,-1.0,IC,745.0,34.0314,-118.3988,2023,4,1,0,20
80531,2020-05-27,1800,13,1375,2,624,52.0,F,H,104.0,...,400.0,IC,624.0,33.9930,-118.2499,2020,5,2,0,18
858965,2023-06-21,800,1,192,2,745,46.0,M,O,122.0,...,-1.0,IC,745.0,34.0347,-118.2646,2023,6,2,0,8


In [5]:
# Hücre 5: Target seçimi, NaN temizliği, one-hot encoding

TARGET_COL_ADI = 'Part 1-2' 

if TARGET_COL_ADI not in df.columns:
    raise ValueError(f"Target kolonu bulunamadı: {TARGET_COL_ADI}")

print("Target değer dağılımı (NaN dahil):")
print(df[TARGET_COL_ADI].value_counts(dropna=False))

# Target NaN olan satırları at
df = df.dropna(subset=[TARGET_COL_ADI])
print("Target NaN temizlendikten sonra satır sayısı:", df.shape[0])

y = df[TARGET_COL_ADI]
X = df.drop(columns=[TARGET_COL_ADI])

print("Feature sayısı (target hariç):", X.shape[1])

# Kategorik kolonlar için one-hot encoding
X = pd.get_dummies(X, drop_first=True)

print("One-hot sonrası feature sayısı:", X.shape[1])
X.head()

Target değer dağılımı (NaN dahil):
Part 1-2
1    48283
2    31717
Name: count, dtype: int64
Target NaN temizlendikten sonra satır sayısı: 80000
Feature sayısı (target hariç): 20
One-hot sonrası feature sayısı: 2146


,TIME OCC,AREA,Rpt Dist No,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,LAT,LON,...,"Premis Desc_VEHICLE, PASSENGER/TRUCK",Premis Desc_VETERINARIAN/ANIMAL HOSPITAL,Premis Desc_VISION CARE FACILITY*,Premis Desc_WAREHOUSE,Premis Desc_WEBSITE,Premis Desc_YARD (RESIDENTIAL/BUSINESS),Status_AO,Status_IC,Status_JA,Status_JO
75319,700,14,1416,310,33.0,501.0,500.0,310.0,34.0100,-118.4497,...,False,False,False,False,False,False,False,True,False,False
655426,1155,1,138,330,24.0,122.0,-1.0,330.0,34.0461,-118.2395,...,True,False,False,False,False,False,False,True,False,False
652607,2051,8,897,745,68.0,502.0,-1.0,745.0,34.0314,-118.3988,...,False,False,False,False,False,False,False,True,False,False
80531,1800,13,1375,624,52.0,104.0,400.0,624.0,33.9930,-118.2499,...,False,False,False,False,False,False,False,True,False,False
858965,800,1,192,745,46.0,122.0,-1.0,745.0,34.0347,-118.2646,...,True,False,False,False,False,False,False,True,False,False


In [6]:
# Hücre 6: Train-test split

print("Target sınıf dağılımı:")
print(y.value_counts())

# Bazı sınıflar çok azsa stratify hata verebilir.
# Hata alırsan 'stratify=y' kısmını silerek tekrar dene.

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Hata olursa bunu kaldır
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

print("y_train NaN sayısı:", y_train.isna().sum())

Target sınıf dağılımı:
Part 1-2
1    48283
2    31717
Name: count, dtype: int64
Train shape: (64000, 2146)
Test shape: (16000, 2146)
y_train NaN sayısı: 0


In [7]:
# Hücre 7: NaN imputing (median ile)

print("NaN sayısı (train, önce):", X_train.isna().sum().sum())
print("NaN sayısı (test,  önce):", X_test.isna().sum().sum())

imputer = SimpleImputer(strategy="median")

X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_imp = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print("NaN sayısı (train, sonra):", X_train_imp.isna().sum().sum())
print("NaN sayısı (test,  sonra):", X_test_imp.isna().sum().sum())

NaN sayısı (train, önce): 1
NaN sayısı (test,  önce): 0
NaN sayısı (train, sonra): 0
NaN sayısı (test,  sonra): 0


In [8]:
# Hücre 8: SelectKBest (Mutual Information) ile 1000 feature'a indir

k_features = min(300, X_train_imp.shape[1])
print("k_features:", k_features)

select_kbest = SelectKBest(score_func=mutual_info_classif, k=k_features)

X_train_k = select_kbest.fit_transform(X_train_imp, y_train)
X_test_k = select_kbest.transform(X_test_imp)

kbest_mask = select_kbest.get_support()
kbest_features = X_train_imp.columns[kbest_mask]

print("SelectKBest sonrası feature sayısı:", X_train_k.shape[1])
print("İlk 20 feature:", kbest_features[:20].tolist())

k_features: 300
SelectKBest sonrası feature sayısı: 300
İlk 20 feature: ['TIME OCC', 'Rpt Dist No', 'Crm Cd', 'Vict Age', 'Premis Cd', 'Weapon Used Cd', 'Crm Cd 1', 'LAT', 'LON', 'year', 'month', 'dayofweek', 'hour', 'DATE OCC_2020-01-14', 'DATE OCC_2020-01-19', 'DATE OCC_2020-01-26', 'DATE OCC_2020-01-27', 'DATE OCC_2020-02-03', 'DATE OCC_2020-02-08', 'DATE OCC_2020-02-18']


In [9]:
# Hücre 9: RFE'yi subset üzerinde çalıştır, 20 feature seç

# 1) X_train_k numpy array, DataFrame'e çevirelim
X_train_k_df = pd.DataFrame(
    X_train_k,
    index=X_train_imp.index,
    columns=kbest_features
)

# 2) RFE için 10.000 satırlık subset alalım
subset_size = min(10000, X_train_k_df.shape[0])  # 10k'dan azsa hepsini alır
X_rfe_train = X_train_k_df.sample(n=subset_size, random_state=42)
y_rfe_train = y_train.loc[X_rfe_train.index]

print("RFE subset shape:", X_rfe_train.shape)

# 3) RFE için daha hafif bir Logistic Regression
rfe_estimator = LogisticRegression(
    max_iter=500,
    multi_class='ovr',      # daha hafif
    solver='liblinear',     # daha hafif solver
    n_jobs=1                # paralel yok, RAM daha az kullanılır
)

rfe = RFE(
    estimator=rfe_estimator,
    n_features_to_select=20,
    step=0.2                # daha hızlı ilerlesin
)

rfe.fit(X_rfe_train, y_rfe_train)

rfe_mask = rfe.get_support()
feat_selected = kbest_features[rfe_mask]

print("RFE sonrası seçilen feature sayısı:", len(feat_selected))
print("Seçilen feature'lar:")
print(feat_selected.tolist())

# 4) Bu 20 feature'ı tüm train/test'e uygula
X_train_fs = X_train_k_df[feat_selected]

X_test_k_df = pd.DataFrame(
    X_test_k,
    index=X_test_imp.index,
    columns=kbest_features
)
X_test_fs = X_test_k_df[feat_selected]

print("Final train shape:", X_train_fs.shape)
print("Final test shape:", X_test_fs.shape)

# Seçilen feature'ları kaydetmek istersen:
feat_selected.to_series().to_csv("selected_features_20.csv", index=False)

RFE subset shape: (10000, 300)


C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


RFE sonrası seçilen feature sayısı: 20
Seçilen feature'lar:
['DATE OCC_2020-01-26', 'DATE OCC_2020-11-15', 'DATE OCC_2021-01-28', 'DATE OCC_2022-04-21', 'DATE OCC_2022-05-02', 'DATE OCC_2022-07-07', 'DATE OCC_2022-10-24', 'DATE OCC_2023-02-10', 'DATE OCC_2023-05-08', 'Vict Sex_UNKNOWN', 'Vict Descent_UNKNOWN', 'Premis Desc_BANK', 'Premis Desc_CLOTHING STORE', 'Premis Desc_DEPARTMENT STORE', 'Premis Desc_LIBRARY', 'Premis Desc_MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)', 'Premis Desc_PARKING UNDERGROUND/BUILDING', 'Premis Desc_PORCH, RESIDENTIAL', 'Premis Desc_SINGLE FAMILY DWELLING', 'Premis Desc_VEHICLE, PASSENGER/TRUCK']
Final train shape: (64000, 20)
Final test shape: (16000, 20)


C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [10]:
# Hücre 10: Final Logistic Regression modelini eğit ve değerlendir

final_log_model = LogisticRegression(
    max_iter=2000,
    multi_class='multinomial',
    solver='saga',
    n_jobs=-1     # Artık sadece 20 feature olduğu için bu daha güvenli
)

final_log_model.fit(X_train_fs, y_train)

y_pred = final_log_model.predict(X_test_fs)

acc = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", acc)
print("F1 (macro):", f1_macro)
print("F1 (weighted):", f1_weighted)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.71725
F1 (macro): 0.690448043613161
F1 (weighted): 0.7093141615965902

Classification Report:
              precision    recall  f1-score   support

           1       0.73      0.84      0.78      9657
           2       0.68      0.53      0.60      6343

    accuracy                           0.72     16000
   macro avg       0.71      0.69      0.69     16000
weighted avg       0.71      0.72      0.71     16000

Confusion Matrix:
[[8092 1565]
 [2959 3384]]
